In [1]:
import os
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

from sklearn.metrics import roc_auc_score

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
DATA_DIR = "/Users/daniil/Coding/PycharmProjects/Education/projects/Neural Networks/datasets/horse-or-human"
BATCH_SIZE = 32
IMG_SIZE = 256

In [4]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.6, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ColorJitter(0.2, 0.2, 0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], # Отклонение RGB канала
                         [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(int(IMG_SIZE * 1.15)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [5]:
train_ds = datasets.ImageFolder(
    os.path.join(DATA_DIR, "train"),
    transform=train_transform
)

val_ds = datasets.ImageFolder(
    os.path.join(DATA_DIR, "validation"),
    transform=val_transform
)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE,
                          shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE,
                        shuffle=False, num_workers=2)

train_ds.classes

['horses', 'humans']

In [6]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 1)
model = model.to(device)

/Users/daniil/.pyenv/versions/3.11.11/envs/ml-env/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/daniil/.pyenv/versions/3.11.11/envs/ml-env/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [8]:
def evaluate_auc(model, loader):
    model.eval()
    probs = []
    targets = []

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device).float()
            logits = model(x).squeeze(1)
            p = torch.sigmoid(logits)

            probs.append(p.cpu().numpy())
            targets.append(y.cpu().numpy())

    probs = np.concatenate(probs)
    targets = np.concatenate(targets)

    return roc_auc_score(targets, probs)

In [9]:
EPOCHS = 5

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for x, y in tqdm(train_loader):
        x = x.to(device)
        y = y.to(device).float().unsqueeze(1)

        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    auc = evaluate_auc(model, val_loader)

    print(f"Epoch {epoch+1}/{EPOCHS} | "
          f"loss = {total_loss/len(train_loader):.4f} | "
          f"ROC-AUC = {auc:.4f}")

100%|██████████| 33/33 [01:25<00:00,  2.58s/it]


Epoch 1/5 | loss = 0.0803 | ROC-AUC = 1.0000


100%|██████████| 33/33 [01:14<00:00,  2.27s/it]


Epoch 2/5 | loss = 0.0034 | ROC-AUC = 0.9999


100%|██████████| 33/33 [01:14<00:00,  2.27s/it]


Epoch 3/5 | loss = 0.0030 | ROC-AUC = 1.0000


100%|██████████| 33/33 [01:21<00:00,  2.47s/it]


Epoch 4/5 | loss = 0.0359 | ROC-AUC = 1.0000


100%|██████████| 33/33 [01:17<00:00,  2.36s/it]


Epoch 5/5 | loss = 0.0168 | ROC-AUC = 1.0000


In [11]:
torch.save(model.state_dict(), "horse_human_resnet18.pth")